In [5]:
import tkinter as tk
from tkinter import ttk
from tkinter.filedialog import askopenfilename
import matplotlib
matplotlib.use('TkAgg')
import model, os
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
class Mainframe(ttk.Frame):
    ###create main frame that will hold all permanent widgets
    def __init__(self, master):
        #mainframe configuration
        ttk.Frame.__init__(self, master)
        self.master = master
        self.padding="3 3 12 12"
        self.grid(column=0, row=0, sticky=(tk.N, tk.W, tk.E, tk.S))
        self.columnconfigure(0, weight=1)
        self.rowconfigure(0, weight=1)
        
        #file parameters
        self.filenamefull = tk.StringVar()
        self.filename = tk.StringVar()
        self.headerlength = tk.IntVar()
        self.datatype = tk.StringVar()
        self.samplingrate = tk.DoubleVar()
        
        #backend interface variables
        self.recording = None
        
        #frame contents
        self.commandbar = Commandframe(self)
        self.commandbar.grid(column=1,row=1,sticky=(tk.E, tk.N, tk.W))
        self.commandbar.load.focus()
        
    def quit(self):
        """
        Close all windows and exit the main loop
        """
        self.master.destroy()
        self.master.quit()
    
class Commandframe(ttk.Frame):
    def __init__(self, parent):
        ttk.Frame.__init__(self, parent)
        self.parent = parent
        self.grid(column=0, row=0, sticky=(tk.N, tk.W, tk.E))
        self.rowconfigure(0, weight=1)
        self.create_widgets()
    def create_widgets(self):
        self.load = ttk.Button(self, text="Load file", command=self.load_dialog)
        self.load.grid(column=1,row=1,sticky=(tk.N, tk.E))
    def load_dialog(self):
        subframe = Loadframe(self.parent)
        
class Plotframe(ttk.Frame):
    def __init__(self, parent):
        ttk.Frame.__init__(self, parent)
        self.parent = parent
        self.grid(column=0, row=0, sticky=(tk.N, tk.W, tk.E))
        self.rowconfigure(0, weight=1)
    
class Loadframe(tk.Toplevel):
    ### temporary frame that gets the file and information about it
    def __init__(self, master):
        tk.Toplevel.__init__(self,master)
        self.master = master
        self.padding="3 3 12 12"
                    
        self.filenamefull = tk.StringVar()
        self.filename = tk.StringVar()
        self.path = tk.StringVar()
        self.filetypefull = tk.StringVar()
        self.filetype = tk.StringVar()
        
        self.create_widgets()
        
        self.loadbutton.focus()

    def create_widgets(self):
        # first row - filename and button for choosing file
        ttk.Label(self, text='File:').grid(column=1, row=1, sticky = (tk.N, tk.W))
        ttk.Label(self, textvariable=self.filename).grid(column=2, row=1, sticky=(tk.N, tk.W, tk.E))
        self.loadbutton = ttk.Button(self, text='Select file', command=self.get_file)
        self.loadbutton.grid(column = 3, row = 1, sticky=(tk.N, tk.E))
        #second row - show filepath
        ttk.Label(self, text='Path:').grid(column=1, row=2, sticky = tk.W)
        ttk.Label(self, textvariable=self.path).grid(column=2, row=2, sticky=(tk.W, tk.E))
        #third row - show filetype
        ttk.Label(self, text='Filetype:').grid(column=1, row=3, sticky = tk.W)
        ttk.Label(self, textvariable=self.filetypefull).grid(column=2, row=3, sticky=(tk.W, tk.E))
        #fourth row - ok button to close and go to next window
        self.okbutton = ttk.Button(self, text="Ok", command=self.ok_button)
        self.okbutton.grid(column=1, row=4, sticky=(tk.S, tk.W))
        self.closebutton = ttk.Button(self, text="Quit", command=self.master.quit)
        self.closebutton.grid(column=3, row=4, sticky=(tk.S, tk.E))
    
    def ok_button(self):
        self.master.filenamefull = self.filenamefull
        self.master.filename = self.filename
        if self.filetype.get() == 'bin':
            binframe = Binaryquery(self.master)
            self.master.recording = model.Recording(self.filenamefull.get(), self.master.samplingrate.get(), self.filetype.get(), self.master.headerlength.get(), self.master.datatype.get())
        elif self.filetype.get() == 'axo':
            axoframe = Axoquery(self.master)
            self.master.recording = model.Recording(self.filenamefull.get(), self.master.samplingrate.get(), self.filetype.get())
        self.destroy()            
    
    def get_file(self):
        """
        Get data by clicking
        relies on tkinter and gets name and type of the file
        """
        fn = askopenfilename()
        self.filenamefull.set(fn)
        extension = ''
        i = 0
        N = len(fn)
        for char in fn[::-1]:
            if char=='.': 
                period = N-i
            if char=='/':
                slash = N-i
                break
            i+=1
        self.filename.set(fn[slash:])
        self.path.set(fn[:slash])
        extension=fn[period:]
        if extension == 'bin':
            self.filetype.set('bin')
            self.filetypefull.set('binary')
        elif extension == 'axgd':
            self.filetype.set('axo')
            self.filetypefull.set('axograph')

class Binaryquery(tk.Toplevel):
    def __init__(self, master):
        #frame configuration
        tk.Toplevel.__init__(self,master)
        self.master = master
        self.padding="3 3 12 12"
        self.title("Additional parameters for binary file")
        
        #initialize content
        self.create_widgets()
    
    def create_widgets(self):
        #get sampling rate
        self.samplingentry = ttk.Entry(self, width=7, textvariable=self.master.samplingrate)
        self.samplingentry.grid(column=3,row=1,sticky=(tk.W,tk.N))
        ttk.Label(self, text="Samplingrate (Hz):").grid(column=2,row=1,sticky=(tk.N))
        #entry field for headerlength
        self.headerentry = ttk.Entry(self, width=7, textvariable=self.master.headerlength)
        self.headerentry.grid(column=3,row=2,sticky=(tk.W,tk.N))
        ttk.Label(self, text="Headerlength:").grid(column=2,row=1,sticky=(tk.N))
        #entry field for filetype
        self.typeentry = ttk.Entry(self, width=7, textvariable=self.master.datatype)
        self.typeentry.grid(column=3,row=3,sticky=(tk.W,tk.S))
        ttk.Label(self, text="Datatype:").grid(column=2,row=2,sticky=tk.S)
        #'ok' and 'cancel button
        self.okbutton = ttk.Button(self, text="Ok", command=self.destroy)
        self.okbutton.grid(column=2, row=4, sticky=(tk.S, tk.W))

class Axoquery(tk.Toplevel):
    def __init__(self, master):
        #frame configuration
        tk.Toplevel.__init__(self,master)
        self.master = master
        self.padding="3 3 12 12"
        self.title("Additional parameters for axograph file")
        
        #initialize content
        self.create_widgets()
    
    def create_widgets(self):
        #get sampling rate
        self.samplingentry = ttk.Entry(self, width=7, textvariable=self.master.samplingrate)
        self.samplingentry.grid(column=3,row=1,sticky=(tk.W,tk.N))
        ttk.Label(self, text="Samplingrate (Hz):").grid(column=2,row=1,sticky=(tk.N))
        #ok button
        self.okbutton = ttk.Button(self, text="Ok", command=self.destroy)
        self.okbutton.grid(column=2, row=2, sticky=(tk.S, tk.W))


In [22]:
def gui_main():
    ### initialize tkinter root window
    root = tk.Tk()
    ### initialize mainframe
    mainframe = Mainframe(root)
    for child in root.winfo_children(): child.grid_configure(padx=5, pady=5)        
    root.mainloop()

In [ ]:
gui_main()

In [11]:
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg, NavigationToolbar2TkAgg
from matplotlib.backend_bases import key_press_handler

In [15]:
x = np.linspace(0,2*np.pi)
y = np.cos(x)
fig = plt.figure(figsize=(5,2))
a = fig.add_subplot(111)
a.plot(x,y)

root = tk.Tk()

canvas = FigureCanvasTkAgg(fig,master=root)
canvas.show()
canvas.get_tk_widget().pack(fill=tk.BOTH,expand=True)

toolbar = NavigationToolbar2TkAgg(canvas, root)
toolbar.update()
canvas._tkcanvas.pack()

In [ ]:
cwd = os.getcwd()
bn = cwd+'/data/sim1600.bin'
bt = np.int16
bh = 3072
rec = model.Recording(filename=filename,filetype='bin',headerlength=bh,bindtype=bt)